In [1]:
import numpy as np
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import classification_report
from collections import Counter
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, fbeta_score

In [2]:
train_data_loaded = pd.read_csv('../data/train_data_2024-08-01.csv')

X = train_data_loaded.drop(columns=['UKATEGORIE'])
y = train_data_loaded['UKATEGORIE']

In [3]:
# StratifiedKFold initialisieren
skf = StratifiedKFold(n_splits=5)

# Variablen für Ergebnisse
fbetas_rf = []
fbetas_random = []
fbetas_mehrheit = []

In [4]:
beta = 2
fbeta_scorer= make_scorer(fbeta_score, beta=beta)

In [5]:
# Balanced Random Forest Modell mit aktualisierten Parametern
bal_rf_model = BalancedRandomForestClassifier(min_samples_leaf=8, n_estimators=344, min_samples_split=400, max_features='log2', max_depth=15, random_state=42, sampling_strategy='all', replacement=True)

rf_scores = cross_val_score(bal_rf_model, X, y, cv=skf, scoring=fbeta_scorer)
print(f'Durchschnittlicher Random Forest F-beta Score (mean) mit unseren Daten: {rf_scores.mean()}')
print(f'Standardabweichung des Random Forest F-beta Score (std) mit unseren Daten: {rf_scores.std()}')

# zufällige und Mehrheitsklassen-Vorhersagen
counter = 1
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Drucke die ersten 10 values von y_train
    print("nur split y_train:", y_train.values[:100])
    # Drucke die ersten 10 values von y_test
    print("nur split y_test:", y_test.values[:100])
    
    # Random Forest Modell trainieren
    bal_rf_model.fit(X_train, y_train)
    y_pred_rf = bal_rf_model.predict(X_test)
    print("Ersten 10 Vorhersagen mit dem Random Forest Modell:", y_pred_rf[:10])
    
    # Zufällige Vorhersagen basierend auf den Klassenwahrscheinlichkeiten erstellen
    class_counts = Counter(y_test)
    total_samples = len(y_test)
    class_probabilities = {cls: count / total_samples for cls, count in class_counts.items()}
    
    np.random.seed(42)
    y_pred_random_weighted = np.random.choice(
        list(class_probabilities.keys()),
        size=y_test.shape,
        p=list(class_probabilities.values())
    )
    
    # Drucke die ersten 10 Werte
    print("Ersten 10 Vorhersagen mit dem _random_weighted:", y_pred_random_weighted[:10])
    
    # Leistung der gewichteten zufälligen Vorhersagen bewerten
    fbeta_random_weighted = fbeta_score(y_test, y_pred_random_weighted, beta=beta, average='weighted')
    print(f"Fold Nummer {counter}: fbeta_random_weighted - Score ist {fbeta_random_weighted}")
    fbetas_random.append(fbeta_random_weighted)
    
    # Immer Mehrheitsklasse vorhersagen
    y_pred_majority = np.full(y_test.shape, y_train.mode().iloc[0])
    
    # Drucke die ersten 10 Werte
    print("Ersten 10 Vorhersagen mit dem y_pred_majority:", y_pred_majority[:10])
    
    # Leistung der Vorhersagen der Mehrheitsklasse bewerten
    fbeta_mehrheit = fbeta_score(y_test, y_pred_majority, beta=beta, average='weighted')
    print(f"Fold Nummer {counter}: fbeta_mehrheit - Score ist {fbeta_mehrheit}")
    fbetas_mehrheit.append(fbeta_mehrheit)
    
    counter += 1

# Durchschnittliche F-beta-Scores berechnen
print(f'Random Weighted Predictions F-beta Score (mean): {np.mean(fbetas_random)}')
print(f'Random Weighted Predictions F-beta Score (std): {np.std(fbetas_random)}')

print(f'Majority Class Predictions F-beta Score (mean): {np.mean(fbetas_mehrheit)}')
print(f'Majority Class Predictions F-beta Score (std): {np.std(fbetas_mehrheit)}')

    

Durchschnittlicher Random Forest F-beta Score (mean) mit unseren Daten: 0.4756496839403724
Standardabweichung des Random Forest F-beta Score (std) mit unseren Daten: 0.00011202336212620579
y_train: [1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0]
y_test: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1
 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 0 0 0
 1 0 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Ersten 10 Vorhersagen mit dem Random Forest Modell: [1 1 1 1 1 1 1 1 1 1]
Ersten 10 Vorhersagen mit dem _random_weighted: [0 1 0 0 0 0 0 1 0 0]
Fold Nummer 1: fbeta_random_weighted - Score ist 0.7463015685247019
Ersten 10 Vorhersagen mit dem y_pred_majority: [0 0 0 0 0 0 0 0 0 0]
Fold Nummer 1: fbeta_mehrheit - Score ist 0.8168620906598428
y_train: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 

In [6]:
# Durchschnittliche Genauigkeiten berechnen
mean_scores_rf = np.mean(fbetas_rf)
mean_scores_random = np.mean(fbetas_random)
mean_scores_mehrheit = np.mean(fbetas_mehrheit)

/home/anne/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/anne/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [7]:
print(f"LogReg Mean score: {mean_scores_rf}")
print(f"Weighted Random Prediction Mean score: {mean_scores_random}")
print(f"Weighted Mehrheit Prediction Mean score: {mean_scores_mehrheit}")


LogReg Mean score: nan
Weighted Random Prediction Mean score: 0.7382213392739827
Weighted Mehrheit Prediction Mean score: 0.8167984428179098


In [8]:

# Klassifikationsberichte für den letzten Fold ausgeben
print("LogReg Classification Report (Last Fold):")
print(classification_report(y_test, y_pred_rf))

print("Weighted Random Prediction Classification Report (Last Fold):")
print(classification_report(y_test, y_pred_random_weighted))

LogReg Classification Report (Last Fold):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      6787
           1       0.15      1.00      0.27      1231

    accuracy                           0.15      8018
   macro avg       0.08      0.50      0.13      8018
weighted avg       0.02      0.15      0.04      8018

Weighted Random Prediction Classification Report (Last Fold):
              precision    recall  f1-score   support

           0       0.84      0.84      0.84      6787
           1       0.14      0.15      0.14      1231

    accuracy                           0.73      8018
   macro avg       0.49      0.49      0.49      8018
weighted avg       0.74      0.73      0.74      8018



/home/anne/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anne/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anne/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
